In [5]:
import os
import gradio as gr
import google.generativeai as genai
from dotenv import load_dotenv
import pandas as pd

# Load environment variables from .env file
load_dotenv()

# Retrieve the API key
api_key = os.getenv("Gemini_API_Key")
if not api_key:
    raise ValueError("API key not found. Ensure 'Gemini_API_Key' is correctly set in the .env file.")

genai.configure(api_key=api_key)

# Load the wine dataset
nlp_file_path = "nlp_3.1_final.csv"
wine_df = pd.read_csv(nlp_file_path)

# Extract unique attribute options
color_options = wine_df["color"].explode().unique().tolist()
aroma_options = wine_df["aromas"].explode().unique().tolist()
flavor_options = wine_df["flavors"].explode().unique().tolist()


def get_wine_recommendation(wine_color, flavor_profile, aroma, sweetness, include_food_pairing):
    """First searches the dataset for a wine recommendation. If no match, use Gemini AI."""
    
    # Filter dataset based on user inputs
    filtered_wines = wine_df[
        wine_df["color"].apply(lambda x: wine_color in x) &
        wine_df["flavors"].apply(lambda x: flavor_profile in x) &
        wine_df["aromas"].apply(lambda x: aroma in x)
    ]
    
    # If we find a matching wine in the dataset
    if not filtered_wines.empty:
        recommended_wine = filtered_wines.sample(1).iloc[0]
        wine_recommendation = f"Recommended Wine: {recommended_wine['color']} - Aromas: {recommended_wine['aromas']} - Flavors: {recommended_wine['flavors']}"
    else:
        # If no exact match, use Gemini AI
        prompt = f"""
        You are a wine expert. Based on the following attributes, recommend a wine:
        
        - Wine Color: {wine_color}
        - Flavor Profile: {flavor_profile}
        - Aroma: {aroma}
        - Sweetness Level: {sweetness}
        
        Provide the best wine recommendation.
        """
        model = genai.GenerativeModel("gemini-pro")
        response = model.generate_content(prompt)
        wine_recommendation = response.text if hasattr(response, "text") else "No response from Gemini API."
    
    # If food pairing is opted in
    if include_food_pairing:
        prompt_food = f"""
        You are a wine and food expert. Based on the following wine attributes, suggest the best food pairings:
        
        - Wine Color: {wine_color}
        - Flavor Profile: {flavor_profile}
        - Aroma: {aroma}
        - Sweetness Level: {sweetness}
        
        Suggest a protein dish, a vegetarian option, and a dessert.
        """
        food_response = model.generate_content(prompt_food)
        food_recommendation = food_response.text if hasattr(food_response, "text") else "No response from Gemini API."
        return f"{wine_recommendation}\n\nFood Pairing: {food_recommendation}"
    
    return wine_recommendation

# Create Gradio UI
demo = gr.Interface(
    fn=get_wine_recommendation,
    inputs=[
        gr.Dropdown(label="Wine Color", choices=color_options),
        gr.Dropdown(label="Flavor Profile", choices=flavor_options),
        gr.Dropdown(label="Aroma", choices=aroma_options),
        gr.Dropdown(label="Sweetness", choices=["Dry", "Off-Dry", "Sweet"]),
        gr.Checkbox(label="Include Food Pairing", value=False)
    ],
    outputs=gr.Textbox(label="Wine Recommendation and Food Pairing"),
    title="🍷 AI Wine & Food Pairing Assistant",
    description="Select wine attributes and get expert recommendations, with an optional food pairing suggestion."
)

demo.launch()

FileNotFoundError: [Errno 2] No such file or directory: 'nlp_3.1_final.csv'